In [1]:
import json as json
import csv as csv
import pandas as pd
import numpy as np
outfile = open("review_stars.tsv", 'w')
sfile = csv.writer(outfile, delimiter ="\t", quoting=csv.QUOTE_MINIMAL) 
sfile.writerow(['business_id','stars', 'text'])
with open('./yelp_dataset/yelp_academic_dataset_review.json', encoding='utf-8') as f: 
    for line in f:
        row = json.loads(line)
        # some special char must be encoded in 'utf-8' 
        sfile.writerow([row['business_id'], row['stars'], (row['text']).encode('utf-8')])
outfile.close()
outfile2 = open("business_dataset.tsv", 'w')
sfile2 = csv.writer(outfile2, delimiter ="\t", quoting=csv.QUOTE_MINIMAL) 
sfile2.writerow(['business_id','review_count', 'categories'])
with open('./yelp_dataset/yelp_academic_dataset_business.json', encoding='utf-8') as f2: 
    for line in f2:
        row = json.loads(line)
        # some special char must be encoded in 'utf-8' 
        if row['categories'] is None:
            sfile2.writerow([row['business_id'], row['review_count']])
        else :
            sfile2.writerow([row['business_id'], row['review_count'], (row['categories']).encode('utf-8')])
outfile2.close()


In [2]:
df1= pd.read_csv('review_stars.tsv', delimiter ="\t", encoding="utf-8",na_values=['NA','?'])
df2= pd.read_csv('business_dataset.tsv', delimiter ="\t", encoding="utf-8",na_values=['NA','?'])


In [3]:
df1.head()


,business_id,stars,text
0,iCQpiavjjPzJ5_3gPD5Ebg,2,"b""The pizza was okay. Not the best I've had. I..."
1,pomGBqfbxcqPv14c3XH-ZQ,5,b'I love this place! My fiance And I go here a...
2,jtQARsP6P-LbkyjbO1qNGg,1,b'Terrible. Dry corn bread. Rib tips were all ...
3,elqbBhBfElMNSrjFqW3now,2,b'Back in 2005-2007 this place was my FAVORITE...
4,Ums3gaP2qM3W1XcA5r6SsQ,5,b'Delicious healthy food. The steak is amazing...


In [4]:
df2.head()

,business_id,review_count,categories
0,Apn5Q_b6Nz61Tq4XzPdf9A,24,"b'Tours, Breweries, Pizza, Restaurants, Food, ..."
1,AjEbIBw6ZFfln7ePHha9PA,3,"b'Chicken Wings, Burgers, Caterers, Street Ven..."
2,O8S5hYJ1SMc8fA4QBtVujA,5,"b'Breakfast & Brunch, Restaurants, French, San..."
3,bFzdJJ3wp3PZssNEsyU23g,8,"b'Insurance, Financial Services'"
4,8USyCYqpScwiNEb58Bt6CA,4,"b'Home & Garden, Nurseries & Gardening, Shoppi..."


In [5]:
# dfa = df1
# dfb = df2
dfa = df1[0:50000]
dfb = df2[0:50000]

In [6]:
dfmerged = pd.merge(dfa, dfb, on='business_id')

In [7]:
df_review_agg = dfmerged.groupby('business_id')['text'].sum()
df_review_agg.sort_index(inplace=True)
dfmerged.stars = dfmerged.stars.astype(float)
df_stars_agg = dfmerged.groupby('business_id')['stars'].mean()
df_stars_agg.sort_index(inplace=True)
df_rating_count = dfmerged.groupby('business_id')['review_count'].mean()
df_rating_count.sort_index(inplace=True)
df_ready_for_sklearn = pd.DataFrame({'business_id': df_review_agg.index, 'all_reviews': df_review_agg.values, 'all_stars' : df_stars_agg.values, 'review_count' : df_rating_count.values})

In [8]:
df_ready_for_sklearn[0:5]

,business_id,all_reviews,all_stars,review_count
0,--6MefnULPED_I942VcFNA,"b""The incredibly rude woman behind the cashier...",1.0,39
1,--Y7NhBKzLTbNliMUX_wfg,b'I have hired the Pinnacle Restoration for 3 ...,5.0,12
2,--g-a85VwrdZJNf0R95GcQ,b'super fresh food..great prices. ala carte an...,5.0,24
3,--orEUqwTzz5QKbmyYbAWw,"b""Hour-long waits are apparently typical and e...",1.0,3
4,-050d_XIor1NpCuWkbIVaQ,"b""One of the best breakfast places in Phoenix....",4.0,698


In [9]:
import sklearn.feature_extraction.text as sk_text
# vectorizer = sk_text.CountVectorizer(min_df=1, stop_words = 'english')
vectorizer = sk_text.TfidfVectorizer(
                             stop_words='english',
                             max_features = 1000,
                             min_df=2, max_df=0.5)
matrix = vectorizer.fit_transform(df_ready_for_sklearn.all_reviews)


In [10]:
tfidf_data = matrix.toarray()

In [11]:
print(tfidf_data)        #  convert it to numpy array

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [12]:
from scipy.stats import zscore

df_ready_for_sklearn['normalize_rc'] = zscore(df_ready_for_sklearn['review_count'])
df_ready_for_sklearn[0:5]


,business_id,all_reviews,all_stars,review_count,normalize_rc
0,--6MefnULPED_I942VcFNA,"b""The incredibly rude woman behind the cashier...",1.0,39,-0.345096
1,--Y7NhBKzLTbNliMUX_wfg,b'I have hired the Pinnacle Restoration for 3 ...,5.0,12,-0.489705
2,--g-a85VwrdZJNf0R95GcQ,b'super fresh food..great prices. ala carte an...,5.0,24,-0.425435
3,--orEUqwTzz5QKbmyYbAWw,"b""Hour-long waits are apparently typical and e...",1.0,3,-0.537909
4,-050d_XIor1NpCuWkbIVaQ,"b""One of the best breakfast places in Phoenix....",4.0,698,3.184447


In [13]:
y = df_ready_for_sklearn['normalize_rc'].values.reshape( len(df_ready_for_sklearn['normalize_rc']),1)

y

array([[-0.34509598],
       [-0.48970546],
       [-0.42543458],
       ...,
       [-0.52184091],
       [-0.53255272],
       [-0.03445338]])

In [14]:
tfidf_data = np.concatenate([tfidf_data,y],axis=1)

In [15]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(tfidf_data,df_ready_for_sklearn['all_stars'], test_size=0.25, random_state=42)

In [16]:
# Linear Regression
from sklearn import datasets, linear_model
linreg = linear_model.LinearRegression()

# fit the model with data
linreg.fit(x_train, y_train)

# predict the response for new observations
linreg.predict(x_test)

array([4.83311911, 4.75546749, 3.59332121, ..., 4.64916021, 4.30080896,
       4.74658304])

In [17]:
# Logistic Regression
from sklearn.linear_model import LogisticRegression

# instantiate the model (using the default parameters)
logreg = LogisticRegression()

# fit the model with data
logreg.fit(x_train, y_train.astype(int))

# predict the response for new observations
logreg.predict(x_test)

array([5, 4, 4, ..., 5, 4, 5])

In [18]:
#correct
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=1)
print(knn)
knn.fit(x_train, y_train.astype('int'))
knn.predict(x_test)  

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=1, p=2,
           weights='uniform')


array([5, 3, 3, ..., 4, 3, 4])

In [19]:
# Nearest Neighbor
from sklearn.neighbors import NearestNeighbors
#X = np.array()
nbrs = NearestNeighbors(n_neighbors=10, algorithm='ball_tree').fit(x_train,x_test)
distances, indices = nbrs.kneighbors(x_train)
indices 


array([[   0,  440, 1414, ..., 5106, 2612, 3917],
       [   1, 5796, 4356, ..., 3159, 4489, 4111],
       [   2,  202, 3954, ..., 3627, 1447, 2039],
       ...,
       [5977, 1110, 1606, ..., 5470, 4602, 1605],
       [5978, 5450, 2157, ..., 3301, 4363, 4676],
       [5979, 4561, 4480, ..., 3266, 4525, 1544]])

In [20]:
#Support Vector Machine
from sklearn import svm
clf = svm.SVC()
clf.fit(x_train, y_train.astype('int'))  
clf.predict(x_test)

array([5, 4, 5, ..., 5, 5, 5])

In [21]:
# Multinomial Naive Bayes
from sklearn import preprocessing
from sklearn.naive_bayes import MultinomialNB
min_max_scaler = preprocessing.MinMaxScaler()
mnb = MultinomialNB()
mnb.fit(min_max_scaler.fit_transform(x_train), y_train.astype(int))
mnb.predict(min_max_scaler.fit_transform(x_test))

array([5, 4, 4, ..., 5, 4, 5])

In [22]:
#Extra : GaussianNB
import numpy as np

from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
clf.fit(x_train, y_train.astype(int))
#MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)
print(clf.predict(x_test))


[5 4 4 ... 5 4 5]
